# Aug 29, 2025: pyvista testing
conda env: pyvista-env

In [1]:
import os 
import numpy as np 
import pandas as pd
import nibabel as nib
import pyvista as pv 
import colorcet as cc
from itertools import product

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()
args.SEED = 100

np.random.seed(args.SEED)

In [3]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.makedirs(PARCELS_path, exist_ok=True)

In [4]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 172
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [5]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'grp'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
TS_path = f'{ROI_path}/roi_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
DIAG_path = f'{ROI_RESULTS_path}/diagnostics'
os.system(f'mkdir -p {DIAG_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [6]:
TEMP_DESC = (
    f'source-allen' #f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-200'

In [7]:
SPACING = np.array([args.resolution] * 3)
ORIGIN = np.array([0] * 3)

In [8]:
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
template_img = nib.load(template_file)

In [9]:
parcels_file = f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
parcels_img = nib.load(parcels_file)

try:
    roi_table = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')
    roi_labels = roi_table['roi'].to_numpy()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [10]:
iso = 0.5                 # shell isosurface for binary template (>0)
pad_vox = (4, 4, 4)       # padding to avoid open ends (olfactory/cerebellum)
upsample_factor = 0.5     # halve voxel size (2× finer); set to None to skip
opacity_shell = 0.18      # glass shell for pretty views (HTML/PNG)
opacity_parc  = 1.0       # opaque for crisp categorical colors
show_scalar_bar = False

In [11]:
def pad_volume(vol, spacing, pad=(4,4,4), origin=(0,0,0), name="scalars"):
    (sx, sy, sz) = spacing
    px, py, pz = pad 
    padw = ((px, px), (py, py), (pz, pz))
    vol_p = np.pad(vol, padw, mode="constant", constant_values=0)
    ox, oy, oz = origin
    origin_p = (ox - px*sx, oy - py*sy, oz - pz*sz)
    
    g = pv.ImageData()
    g.dimensions = vol_p.shape
    g.spacing    = spacing
    g.origin     = origin_p
    g.point_data[name] = vol_p.reshape(-1, order="F")
    return g

def make_shell(grid: pv.ImageData, iso=0.5, sample_rate=2.0, smooth_iters=100, decimate=0.5):
    g = grid
    g = g.resample(sample_rate=sample_rate)
    shell = g.contour([iso], method='marching_cubes').smooth_taubin(n_iter=smooth_iters)
    if decimate is not None and 0.0 < decimate < 1.0:
        shell = shell.decimate(decimate)
    return shell

In [12]:
img = nib.load(template_file)
template_vol = img.get_fdata()[::-1, :, :]
spacing = img.header.get_zooms()[:3]
template_grid = pad_volume(template_vol, spacing, pad=pad_vox, origin=(0, 0, 0), name='template')
template_mesh = make_shell(template_grid, iso=28, sample_rate=1, smooth_iters=75)

In [13]:
x0 = 0.5*(template_mesh.bounds[0] + template_mesh.bounds[1])
y0 = 0.5*(template_mesh.bounds[2] + template_mesh.bounds[3])
z0 = 0.5*(template_mesh.bounds[4] + template_mesh.bounds[5])
MID_ORIGIN = (x0, y0, z0)

In [14]:
R_SHELL = template_mesh.clip(normal=(-1,0,0), origin=MID_ORIGIN).triangulate().clean()  # keeps x >= x0
L_SHELL = template_mesh.clip(normal=( 1,0,0), origin=MID_ORIGIN).triangulate().clean()  # keeps x <= x0
R_SHELL = R_SHELL.fill_holes(50).clean()
L_SHELL = L_SHELL.fill_holes(50).clean()

In [15]:
template_df = pd.DataFrame(dict(
    hemi=['L', 'R', 'B'],
    mesh=[L_SHELL, R_SHELL, template_mesh],
))
template_df

,hemi,mesh
0,L,PolyData (0x7f57395f25c0)\n N Cells: 11106...
1,R,PolyData (0x7f57395f2620)\n N Cells: 11362...
2,B,PolyData (0x7f57395f2560)\n N Cells: 20474...


In [16]:
parcels_img = nib.load(parcels_file)
parcels_vol = parcels_img.get_fdata()[::-1, :, :]
# parcels_vol = parcels_img.get_fdata()[::-1, ::-1, :].transpose(0, 2, 1)

rois_df = []
for roi in [1, 87]: #[28, 29, 30, 33, 34, 38, 39, 40, 41, 114, 115, 116, 119, 120, 124, 125, 126, 127]:
    roi_vol = (parcels_vol == roi).astype(int)
    spacing = parcels_img.header.get_zooms()[:3]
    roi_grid = pad_volume(roi_vol, spacing, pad=pad_vox, origin=(0, 0, 0), name=f'roi-{roi:03d}')
    roi_mesh = make_shell(roi_grid, sample_rate=1, smooth_iters=150)
    row = {
        'roi': roi,
        'hemi': 'R' if roi <= args.num_rois / 2 else 'L',
        'mesh': roi_mesh,
        'color': np.random.random(3),
    }
    rois_df += [row]
rois_df = pd.DataFrame(rois_df).sort_values('roi').reset_index(drop=True)
rois_df

,roi,hemi,mesh,color
0,1,R,PolyData (0x7f57395f3e80)\n N Cells: 190\n...,"[0.5434049417909654, 0.27836938509379616, 0.42..."
1,87,L,PolyData (0x7f57395f3fa0)\n N Cells: 160\n...,"[0.8447761323199037, 0.004718856190972565, 0.1..."


In [17]:
sub = 'SLC01'
SBM = 'sbm-nd-h'
mode_id = 0
level = 0

In [18]:
folder = (
    f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
    f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
)

mode_file = f'{folder}/mode-{mode_id:02d}_level-{level}.nii.gz'

In [19]:
views_df = pd.DataFrame(dict(
    idx=[0, 1, 2, 3, 4, 5, 6, 7],
    hemi=['L', 'B', 'B', 'B', 'R', 'B', 'B', 'B']
))
views_df

,idx,hemi
0,0,L
1,1,B
2,2,B
3,3,B
4,4,R
5,5,B
6,6,B
7,7,B


In [20]:
camera_args = dict(
    pos='yz',
    roll=180,
    a=0, # update azimuth here for each view
    e=35,
)

In [21]:
def add_RIA_axes(p: pv.Plotter):
    ax = p.add_axes(line_width=1)
    ax.SetXAxisLabelText('R')
    ax.SetYAxisLabelText('I')
    ax.SetZAxisLabelText('A')
    ax.SetTotalLength(2, 2, 2)
    return ax

In [22]:
def select_rois(roi_df, hemi='B'):
    if hemi == 'B':
        return roi_df
    else:
        return roi_df[roi_df['hemi'] == hemi].reset_index(drop=True)

In [23]:
def plot_view(p: pv.Plotter, idx, cam_args=dict(), ):
    hemi = views_df.at[idx, 'hemi']
    underlay_mesh = template_df[template_df['hemi'] == hemi]['mesh'].to_list()[0]
    roi_df = select_rois(rois_df, hemi)
    
    # add_RIA_axes(p)
    
    p.add_mesh(
        underlay_mesh,
        color='#cccccc', opacity=0.18, smooth_shading=True,
    )
    
    for idx_, row in roi_df.iterrows():
        roi, hemi, roi_mesh, color = row
        color = '#cccccc' # comment this line after testing
        p.add_mesh(
            roi_mesh, 
            color=color,
            opacity=0.18, #1.0, 
            smooth_shading=True,
        )
        
    p.camera_position = cam_args['pos']
    p.camera.roll = cam_args['roll']
    p.camera.azimuth = cam_args['a']
    p.camera.elevation = cam_args['e']
    
    return p

In [24]:
def single_view(idx, folder):
    ncols = 1
    nrows = 1
    w = 20 / 9 # points
    h = w
    p = pv.Plotter(
        shape=(nrows, ncols),
        # window_size=(ncols*w, nrows*h), 
        off_screen=True, 
        border=False,
    )
    # p.enable_parallel_projection() # identical scale in every subplot
    p.set_background('white')

    camera_args['a'] = 45 * idx
    p = plot_view(p, idx, camera_args)
    
    p.save_graphic(f'{folder}/scene_{idx}.svg')
    p.export_html(f'{folder}/scene.html')

In [25]:
folder = './scenes'
os.system(f'mkdir -p {folder}')
for idx in range(0, 8):
    single_view(idx, folder)

2025-10-15 14:41:28.313 (   1.892s) [    7F581FC7F740]vtkXOpenGLRenderWindow.:1416  WARN| bad X server connection. DISPLAY=
libEGL warning: failed to open /dev/dri/renderD129: Permission denied

libEGL warning: failed to open /dev/dri/card1: Permission denied

2025-10-15 14:41:28.318 (   1.897s) [    7F581FC7F740] vtkEGLRenderWindow.cxx:353   WARN| vtkEGLRenderWindow (0x564a1ff095d0): EGL device index: 0 could not be initialized. Trying other devices...
libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: failed to open /dev/dri/card0: Permission denied

libEGL warning: failed to open /dev/dri/renderD129: Permission denied

libEGL warning: failed to open /dev/dri/card1: Permission denied

2025-10-15 14:41:28.358 (   1.938s) [    7F581FC7F740] vtkEGLRenderWindow.cxx:353   WARN| vtkEGLRenderWindow (0x564a200210c0): EGL device index: 0 could not be initialized. Trying other devices...
libEGL warning: failed to open /dev/dri/renderD128: Permission denied



In [26]:
from pathlib import Path 
import base64, io 
import numpy as np 
from PIL import Image 
import xml.etree.ElementTree as ET 

SVG_NS = 'http://www.w3.org/2000/svg'
XLINK_NS = 'http://www.w3.org/1999/xlink'
ET.register_namespace('', SVG_NS)
ET.register_namespace('xlink', XLINK_NS)

def _get_image_elem(root):
    # Return the first <image> element
    return root.find(f".//{{{SVG_NS}}}image")

def _load_rgba_from_href(svg_path, href):
    if href.startswith("data:image"):
        b64 = href.split(",", 1)[1]
        return Image.open(io.BytesIO(base64.b64decode(b64))).convert("RGBA")
    # relative/absolute file path
    p = Path(href)
    if not p.exists():
        p = (Path(svg_path).parent / href).resolve()
    return Image.open(p).convert("RGBA")

def _crop_white(im, white_threshold=245, pad_px=0):
    a = np.array(im)
    rgb, alpha = a[..., :3], a[..., 3]
    near_white = (rgb >= white_threshold).all(axis=-1)
    a[..., 3] = np.where(near_white, 0, alpha)   # make near-white transparent
    alpha2 = a[..., 3]
    ys, xs = np.where(alpha2 > 0)
    if xs.size == 0:  # nothing found: return original
        return Image.fromarray(a, "RGBA")
    x0, x1 = max(0, xs.min()-pad_px), min(im.width,  xs.max()+1+pad_px)
    y0, y1 = max(0, ys.min()-pad_px), min(im.height, ys.max()+1+pad_px)
    return Image.fromarray(a, "RGBA").crop((x0, y0, x1, y1))

def _write_tight_svg(out_path, pil_rgba):
    buf = io.BytesIO()
    pil_rgba.save(buf, format="PNG")
    data_uri = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode("ascii")
    svg = ET.Element(f"{{{SVG_NS}}}svg", {
        "version":"1.1",
        "width":  f"{pil_rgba.width}px",
        "height": f"{pil_rgba.height}px",
        "viewBox": f"0 0 {pil_rgba.width} {pil_rgba.height}",
    })
    ET.SubElement(svg, f"{{{SVG_NS}}}image", {
        f"{{{XLINK_NS}}}href": data_uri,
        "x":"0", "y":"0",
        "width": str(pil_rgba.width),
        "height":str(pil_rgba.height),
        "preserveAspectRatio":"none",
    })
    ET.ElementTree(svg).write(out_path, encoding="utf-8", xml_declaration=True)
    
def trim_svg(svg_in, svg_out=None, white_threshold=245, pad_px=0):
    """Read SVG, crop embedded raster to remove white margins, write tight SVG."""
    svg_in = Path(svg_in)
    if svg_out is None:
        svg_out = svg_in.with_suffix("").with_name(svg_in.stem + "_cropped").with_suffix(".svg")
    root = ET.parse(svg_in).getroot()
    img = _get_image_elem(root)
    if img is None:
        raise RuntimeError("No <image> element found (this expects an SVG that wraps a raster).")
    href = img.get(f"{{{XLINK_NS}}}href") or img.get("href")
    if not href:
        raise RuntimeError("No href on <image> (neither xlink:href nor href).")
    im = _load_rgba_from_href(svg_in, href)
    cropped = _crop_white(im, white_threshold=white_threshold, pad_px=pad_px)
    _write_tight_svg(svg_out, cropped)
    return str(svg_out), (cropped.width, cropped.height)

In [27]:
from glob import glob 
pad = 10 # px
sizes = []
for svg_in in sorted(glob(f'{folder}/scene_?.svg')):
    svg_out, size = trim_svg(svg_in)
    sizes += [(size[0]+pad, size[1]+pad)]
print(sizes)

[(675, 355), (532, 402), (414, 452), (532, 400), (674, 354), (519, 407), (442, 439), (518, 411)]


In [28]:
from svgutils.compose import Figure, SVG
from pathlib import Path

def montage_scenes(folder, sizes):
    slots = [(1,2), (2,2), (2,1), (2,0), (1,0), (0,0), (0,1), (0,2)]
    
    Ws = np.zeros((3, 3))
    Hs = np.zeros((3, 3))

    for idx in range(8):
        w, h = sizes[idx]
        r, c = slots[idx]
        Ws[r, c] = w 
        Hs[r, c] = h

    # total size of image
    W = np.sum(Ws, axis=1).max() # total width of image
    H = np.sum(Hs, axis=0).max() # total height of image
    
    # positions of each panel
    Xs = np.zeros((3, 3))
    for i in range(3):
        Xs[i,0] = 0
        Xs[i,1] = Ws[i,0]
        Xs[i,2] = W - Ws[i,2]

    Ys = np.zeros((3, 3))
    for j in range(3):
        Ys[0,j] = 0
        Ys[1,j] = Hs[0,j]
        Ys[2,j] = H - Hs[2,j]
        
    elems = [SVG(f'{folder}/scene_{i}_cropped.svg').move(Xs[r,c], Ys[r,c])
            for i,(r,c) in enumerate(slots)]
    Figure(f"{W}px", f"{H}px", *elems).save(f'{folder}/montage.svg')

In [29]:
montage_scenes(folder, sizes)